In [ ]:
#Import Libraries
import pymc as pm
import numpy as np
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
from PIL import Image
import csv
import os

In [ ]:
import matplotlib
# Check versions
print("pymc version:", pm.__version__)
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)
print("arviz version:", az.__version__)
print("matplotlib version:", matplotlib.__version__)# Now access __version__ from matplotlib
print("pymc version:", pm.__version__)
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)
print("arviz version:", az.__version__)
!python --version

pymc version: 5.21.2
numpy version: 2.0.2
pandas version: 2.2.2
arviz version: 0.21.0
matplotlib version: 3.10.0
pymc version: 5.21.2
numpy version: 2.0.2
pandas version: 2.2.2
arviz version: 0.21.0
Python 3.11.11


In [ ]:
def ordinal_predictor_binary_outcome_model_b02(predictor, outcome):
    """
    PyMC model with a binary outcome and an ordinal predictor with a normal random walk constraint,
    with fluctuations around an estimated effect.
    Also includes PPCs, WAIC, and BIC calculation.
    """
    with pm.Model() as model:
        # Define the 'obs' dimension
        n_obs = len(outcome)
        model.add_coord("obs", np.arange(n_obs))

        # Priors
        intercept = pm.Normal("intercept", mu=0, sigma=10)
        effect = pm.Normal("effect", mu=0, sigma=2)
        sd_fluctuation = pm.HalfCauchy("sd_fluctuation", beta=2)

        # Level effects differences
        level_effects_diff = pm.Normal("level_effects_diff", mu=effect, sigma=sd_fluctuation, shape=4)

        # Cumulative level effects
        level_effects = pm.Deterministic("level_effects", pm.math.concatenate([[0], pm.math.cumsum(level_effects_diff)]))

        # Linear predictor
        logit_p = intercept + level_effects[predictor - 1]
        p = pm.Deterministic("p", pm.math.sigmoid(logit_p), dims="obs")  # Dimension 'p' by 'obs'

        # Likelihood - Associate observed data with 'obs' dimension
        y_obs = pm.Bernoulli("y_obs", p=p, observed=outcome, dims="obs")

        # Posterior predictive samples
        y_pred = pm.Bernoulli("y_pred", p=p, observed=None, dims="obs")

        # Sampling - Request log_likelihood
        idata = pm.sample(
            2000,
            tune=1000,
            return_inferencedata=True,
            target_accept=0.9,  # Increased target_accept
            idata_kwargs={"log_likelihood": True}  # Request log_likelihood
        )
        idata.extend(pm.sample_posterior_predictive(idata))

        waic = az.waic(idata)
        loo = az.loo(idata)

        # BIC calculation
        n_params = len(model.free_RVs)
        log_likelihood = idata.log_likelihood.y_obs.values
        bic = -2 * np.mean(log_likelihood) + n_params * np.log(n_obs)  # Use n_obs


    return model, idata, waic, loo, bic


In [ ]:
def run_and_summarize(data, model_func, label="b02", output_dir="results"):
    """
    Runs the model, summarizes results, saves outputs: idata, summary, metrics, plots.
    Works for B01 or B02 depending on the model_func provided.
    """
    os.makedirs(output_dir, exist_ok=True)

    model, idata, waic, loo, bic = model_func(data["predictor"], data["outcome"])

    # Save InferenceData
    idata_path = os.path.join(output_dir, f"idata_{label}.nc")
    idata.to_netcdf(idata_path)

    # Determine var_names based on model
    try:
        # Check if variables like "effect" and "sd_fluctuation" exist (i.e. B02)
        var_names = ["intercept", "effect", "sd_fluctuation", "level_effects", "p"]
        _ = idata.posterior["effect"]
    except KeyError:
        # B01 fallback
        var_names = ["intercept", "level_effects", "p"]

    # Save Summary
    summary = az.summary(idata, var_names=var_names)
    summary_path = os.path.join(output_dir, f"summary_{label}.csv")
    summary.to_csv(summary_path)

    # Save WAIC, LOO, BIC as a markdown file
    metrics_path = os.path.join(output_dir, f"model_metrics_{label}.md")
    with open(metrics_path, "w") as f:
        f.write(f"## Model {label.upper()} Metrics\n\n")
        f.write("### WAIC\n")
        f.write(waic.to_string() + "\n\n")
        f.write("### LOO\n")
        f.write(loo.to_string() + "\n\n")
        f.write(f"### BIC\nBIC = {bic:.2f}\n")

    # Save Plots
    trace_plot_path = os.path.join(output_dir, f"trace_{label}.png")
    az.plot_trace(idata, var_names=[vn for vn in var_names if vn != "p"])
    plt.tight_layout()
    plt.savefig(trace_plot_path, dpi=300)
    plt.close()

    posterior_plot_path = os.path.join(output_dir, f"posterior_p_{label}.png")
    az.plot_posterior(idata, var_names=["p"])
    plt.tight_layout()
    plt.savefig(posterior_plot_path, dpi=300)
    plt.close()

    ppc_plot_path = os.path.join(output_dir, f"ppc_{label}.png")
    az.plot_ppc(idata)
    plt.tight_layout()
    plt.savefig(ppc_plot_path, dpi=300)
    plt.close()

    print(f"✅ Saved all outputs for model {label.upper()} in '{output_dir}' folder.")

    return idata, waic, loo, bic


In [ ]:
# Data Generation
def generate_test_data(n_per_level=100, random_seed=None):
    rng = np.random.default_rng(random_seed)
    predictor_levels = np.repeat(np.arange(1, 6), n_per_level)
    probabilities = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
    true_probabilities = probabilities[predictor_levels - 1]
    outcomes = rng.binomial(1, true_probabilities)
    return pd.DataFrame({"predictor": predictor_levels, "outcome": outcomes})


In [ ]:
# Generate test data
test_data = generate_test_data(random_seed=42)

# Run and save results for B02
print("\nResults for B02 Model:")
idata_b02, waic_b02, loo_b02, bic_b02 = run_and_summarize(test_data, ordinal_predictor_binary_outcome_model_b02, label="b02")


Sampling ... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 / 0:00:01

/usr/local/lib/python3.11/dist-packages/arviz/stats/diagnostics.py:596: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/usr/local/lib/python3.11/dist-packages/arviz/stats/diagnostics.py:991: RuntimeWarning: invalid value encountered in scalar divide
  varsd = varvar / evar / 4
/usr/local/lib/python3.11/dist-packages/arviz/stats/density_utils.py:488: UserWarning: Your data appears to have a single value or no finite values
  warnings.warn("Your data appears to have a single value or no finite values")
/usr/local/lib/python3.11/dist-packages/arviz/plots/plot_utils.py:270: UserWarning: rcParams['plot.max_subplots'] (40) is smaller than the number of variables to plot (500) in plot_posterior, generating only 40 plots
  warnings.warn(
<ipython-input-15-ac14f60c543d>:53: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()


✅ Saved all outputs for model B02 in 'results' folder.


In [ ]:
from google.colab import files
!zip -r results_b02.zip results/
#files.download("results_b02.zip")

In [ ]:
# Make sure you're in the right repo folder
%cd Bayesian-application-to-path-analysis/scripts/

# Upload the new clean notebook file
from google.colab import files
uploaded = files.upload()


[Errno 2] No such file or directory: 'Bayesian-application-to-path-analysis/scripts/'
/content


KeyboardInterrupt: 

In [ ]:
!git add Milestone_B02 (1).ipynb
!git commit -m "Re-uploaded valid notebook"
!git push


/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `git add Milestone_B02 (1).ipynb'
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!git commit -m "Add the Bayesian model script to scripts folder"

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@f960fe968a20.(none)')


In [ ]:
# Configure Git (only if not done yet)
!git config --global user.email "ogunjounb.o@gmail.com"
!git config --global user.name "Mojisibe"

In [ ]:
from getpass import getpass

# Securely enter your GitHub personal access token
token = getpass('Enter your GitHub personal access token: ')

# Replace with your GitHub username and repo name
username = "your-username"
repo_name = "your-private-repo"

# Correctly formatted HTTPS URL
repo_url = f"https://{token}@github.com/{username}/{repo_name}.git"

# Clone the repo
!git clone {repo_url}


In [ ]:
from getpass import getpass

# Paste your personal access token here
token = getpass('Enter your GitHub personal access token: ')

# Replace with your repo info
username = "Mojisibe"
repo_name = "Bayesian-application-to-path-analysis"
repo_url = f"https://{token}@github.com/{username}/{repo_name}.git"

!git clone {https://github.com/Mojisibe/Bayesian-application-to-path-analysis}


Enter your GitHub personal access token: ··········
Cloning into 'Bayesian-application-to-path-analysis}'...
fatal: protocol '{https' is not supported


In [ ]:
!git push origin main

Everything up-to-date


In [ ]:
mv scripts/'Bayesian model with test data.py' scripts/Milestone_B02.ipynb

In [ ]:
!git add scripts/Milestone_B02.ipynb

In [ ]:
!git commit -m "Rename and add the Bayesian model script with the correct name"

[main d065b92] Rename and add the Bayesian model script with the correct name
 1 file changed, 0 insertions(+), 0 deletions(-)
 rename scripts/{Bayesian model with test data.py => Milestone_B02.ipynb} (100%)


In [ ]:
!git push origin main

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 337 bytes | 337.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/Mojisibe/Bayesian-application-to-path-analysis.git
   6c0e22c..d065b92  main -> main


In [ ]:
# Copy your main script
!cp "/content/Bayesian-application-to-path-analysis/Bayesian model with test data.py" "/content/Bayesian-application-to-path-analysis/scripts/"

# Copy the entire 'results' folder
!cp -r /content/results /content/Bayesian-application-to-path-analysis/

In [ ]:
%cd /content/Bayesian-application-to-path-analysis

# Configure Git (only if not done yet)
!git config --global user.email "ogunjounb.o@gmail.com"
!git config --global user.name "Mojisibe"

# Stage changes
!git add .

# Commit with a message
!git commit -m "Add B02 model script and results"

# Push to GitHub
!git push origin main


/content/Bayesian-application-to-path-analysis
[main 993ef02] Add B02 model script and results
 7 files changed, 616 insertions(+)
 create mode 100644 results/idata_b02.nc
 create mode 100644 results/model_metrics_b02.md
 create mode 100644 results/posterior_p_b02.png
 create mode 100644 results/ppc_b02.png
 create mode 100644 results/summary_b02.csv
 create mode 100644 results/trace_b02.png
 create mode 100644 scripts/Bayesian model with test data.py
Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 2 threads
Compressing objects: 100% (10/10), done.
Writing objects: 100% (10/10), 7.39 MiB | 4.71 MiB/s, done.
Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/Mojisibe/Bayesian-application-to-path-analysis.git
   f15a66c..993ef02  main -> main
